In [1]:
import pandas as pd

import re
from emoji import UNICODE_EMOJI
from textblob import TextBlob
import altair as alt
import numpy as np
from collections import Counter
import string

import nltk
nltk.download('vader_lexicon')
nltk.download('brown')
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### The data cleaning/manipulation technique/functions

In [2]:
def extract_tags(text):
    return re.findall("#([a-zA-Z0-9_]{1,50})", text)
    
def extract_emoji(text):
    return [ch for ch in text if ch in UNICODE_EMOJI['en']]


def clean_tweet(txt): 
    temp = re.sub("@[A-Za-z0-9_]+","", txt)
    temp1 = re.sub("#[A-Za-z0-9_]+","", temp)
    temp2 = re.sub(r"http\S+", "", temp1)
    
    result=''.join(i for i in temp2.lower() if (i.isalpha() or i==' '))
    return result

def word_list(tweet):
    
    lst = word_tokenize(tweet)
    lst1 = []
    stops = list(stopwords.words('english'))
    for w in lst:
        if w not in stops:
            lst1.append(w)
    
    return lst1

def sentiment(tweet):
    blob = TextBlob(tweet)
    
    return blob.sentiment.polarity


def get_date(date):
    
    return date[:10]

def get_hour(date):
    
    return date[11:13]
def get_10min(date):
    
    return date[14]+'0'

def get_min(date):
    
    return date[14:16]

def firm_pos(score):
    if score >= 0.7:
        return 1
    else: return 0
    
def pos(score):
    if (score >= 0.25) & (score < 0.7):
        return 1
    else: return 0
    
def neutral(score):
    if (score >= -0.25) & (score < 0.25):
        return 1
    else: return 0
    
def neg(score):
    if (score > -0.7) & (score < -0.25):
        return 1
    else: return 0
    
def firm_neg(score):
    if score <= -0.7:
        return 1
    else: return 0

### **Import data, check duplicate or missing value, remove if exists.**

In [3]:
df = pd.read_csv('Project Data/Lebron 2020 playoff.csv')

df['id'].duplicated(keep='last').sum()

0

In [4]:
df.isnull().sum()

id      0
date    1
text    1
dtype: int64

In [5]:
# drop row with missing value and reset index

df = df.dropna(how='any').reset_index()
df.drop(columns=['index'],inplace=True)

### Apply data cleaning/manipulation techniques on the data, we now have the used words, tags, emojis, sentiment score, and specific date/hour/min data. 

In [6]:
df['tags']= df.apply(lambda row: extract_tags(row['text']), axis=1)
df['emojis']= df.apply(lambda row: extract_emoji(row['text']), axis=1)
df['clean_text']= df.apply(lambda row: clean_tweet(row['text']), axis=1)
df['words']= df.apply(lambda row: word_list(row['clean_text']), axis=1)
df['sentiment_score']= df.apply(lambda row: sentiment(row['clean_text']), axis=1)
df['day']= df.apply(lambda row: get_date(row['date']), axis=1)
df['hour']= df.apply(lambda row: get_hour(row['date']), axis=1)
df['10min']= df.apply(lambda row: get_10min(row['date']), axis=1)
df['min']= df.apply(lambda row: get_min(row['date']), axis=1)
df['POS']= df.apply(lambda row: firm_pos(row['sentiment_score']), axis=1)
df['pos']= df.apply(lambda row: pos(row['sentiment_score']), axis=1)
df['neu']= df.apply(lambda row: neutral(row['sentiment_score']), axis=1)
df['neg']= df.apply(lambda row: neg(row['sentiment_score']), axis=1)
df['NEG']= df.apply(lambda row: firm_neg(row['sentiment_score']), axis=1)

df.head()

,id,date,text,tags,emojis,clean_text,words,sentiment_score,day,hour,10min,min,POS,pos,neu,neg,NEG
0,1295510581181480960,2020-08-17 23:59:26+00:00,@Chris_Auther @KingJames @AntDavis23 I need th...,[],[],i need that,[need],0.0,2020-08-17,23,50,59,0,0,1,0,0
1,1295510388411334656,2020-08-17 23:58:40+00:00,25 hours out.. what time you activating? #Zero...,"[ZeroDark23, GoatMode]","[👑, 🐐]",hours out what time you activating,"[hours, time, activating]",0.0,2020-08-17,23,50,58,0,0,1,0,0
2,1295510348775096320,2020-08-17 23:58:31+00:00,@bigreggie85 @Lakers @Keefmorris @KingJames @k...,[],[],are you a boy,[boy],0.0,2020-08-17,23,50,58,0,0,1,0,0
3,1295510345268830208,2020-08-17 23:58:30+00:00,"@DeanDTD Lebron James, Michael Jackson, Bernie...",[],[],lebron james michael jackson bernie mac post ...,"[lebron, james, michael, jackson, bernie, mac,...",0.0,2020-08-17,23,50,58,0,0,1,0,0
4,1295510271163805699,2020-08-17 23:58:12+00:00,@washingtonpost I really don't want to hear an...,[],[],i really dont want to hear anymore from micha...,"[really, dont, want, hear, anymore, michael, b...",0.2,2020-08-17,23,50,58,0,0,1,0,0


### **Flow of Tweet count & sentiment**

In [7]:
score = df.groupby(['day','hour']).agg([np.size,np.sum]).sentiment_score.reset_index()
score[['10hr_count','10hr_senti']] = score.rolling(window=10,min_periods=1).sum()[['size','sum']]
score['avg'] = score['10hr_senti'] / score['10hr_count']

score['date'] = score['day'] + ' ' + score['hour'] + ":" + '00'
score.head()

,day,hour,size,sum,10hr_count,10hr_senti,avg,date
0,2020-08-17,00,149.0,15.605739,149.0,15.605739,0.104737,2020-08-17 00:00
1,2020-08-17,01,130.0,9.961459,279.0,25.567198,0.091639,2020-08-17 01:00
2,2020-08-17,02,140.0,9.248398,419.0,34.815596,0.083092,2020-08-17 02:00
3,2020-08-17,03,136.0,9.632750,555.0,44.448347,0.080087,2020-08-17 03:00
4,2020-08-17,04,95.0,7.217172,650.0,51.665518,0.079485,2020-08-17 04:00


In [8]:
alt.Chart(score).mark_line().encode(
    x=alt.X('date:T',title='Date'),
    y=alt.Y('size:Q',title='Tweet Count (by hour)')
).properties(width=800,height=480,title='Tweet count flow about Lebron during the 2020 Playoffs')

alt.Chart(...)

In [9]:
senti_line = alt.Chart(score).mark_line().encode(
    x=alt.X('date:T',title='Date'),
    y=alt.Y('avg:Q',title='Avg Sentiment Score')
).properties(width=840,height=480,title='Sentiment Flow about Lebron during 2020 Playoffs')
senti_line

alt.Chart(...)

In [10]:
annotations = [['2020-08-24 12:00:00',0.165, 'Players Strike for Social Justice Issue'],
               ['2020-08-24 12:00:00',0.15, 'All Playoff Games & Activity Suspended'],
               ['2020-09-11 00::00',0.23, 'Lakers Advance to WCF'],
               ['2020-09-27 00:00:00',0.15, 'Lakers Advance to Finals']]
a_df = pd.DataFrame(annotations, columns=['date','values','note'])
a_df

text=alt.Chart(a_df).encode(
    x=alt.X('date:T'),
    y=alt.Y('values:Q'),
    text='note').mark_text(size=12,fontWeight='bold')

(senti_line + text).properties(
    width=840,height=480,
    title={
      "text": ["Sentiment Flow - Lebron 2020 Playoffs"], 
      "subtitle": ["Overall sentiment flow on Twitter about Lebron during the 2020 playoffs, computed by 10 hour rolling average"],
      "color": "black",
      "subtitleFontSize":15
    }).configure_axis(
    labelFontSize=12,
    titleFontSize=20
).configure_title(
    anchor='start',
    fontSize = 24)

alt.LayerChart(...)

## **Emoji/Tag**

In [11]:
# this return the top 50 most common items in the columns (emoji/tag/word)

def top_item(data,label):
    
    lst = []
    for i in data[label]:
        lst += i
        
    C = Counter(lst)
    top50 = C.most_common(50)
    count_df = pd.DataFrame(top50,columns = [label,'count'])
    
    return count_df

In [12]:
c= top_item(df, 'tags')
c

# the top 50 most frequently used tags      within the 'lebron james' tweets during the 2020 playoffs

,tags,count
0,LeBronJames,7114
1,LakersMoment,5844
2,ULTRAmoment,5830
3,NBA,5096
4,LakeShow,4837
5,NBAFinals,4373
6,NBAPlayoffs,4043
7,Lakers,3205
8,nba,2282
9,lebronjames,1663


### **Visualize bar chart for the most popular tags**

In [13]:
c1=c[:25]

alt.Chart(c1).mark_bar().encode(
    x=alt.X('tags',sort=['count'],title='Tags',axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count',title='Total Usage')
).properties(width=900,height=400,title={
      "text": ["Most popular tags - Lebron during 2020 Playoffs"],
      "subtitle":["The top 25 most popular emojis used among the tweets about Lebron during Playoffs"]  
    }).configure_axis(
    labelFontSize=16,
    titleFontSize=20
).configure_title(
    anchor='start',
    fontSize = 24,
    subtitleFontSize = 15
)

alt.Chart(...)

### **Emoji**

In [14]:
count = top_item(df,'emojis')
count

,emojis,count
0,😂,28509
1,🤣,13599
2,🔥,8136
3,👑,7480
4,🏾,7471
5,♂,6492
6,👍,6240
7,🐐,5330
8,🏽,4824
9,😭,4673


In [15]:
c1=count[:30]

alt.Chart(c1).mark_bar().encode(
    x=alt.X('emojis',sort=['count'],title='Emojis',axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count',title='Total Usage')
).properties(width=900,height=400,title={
      "text": ["Most popular emojis - Lebron 2020 Playoffs"],
      "subtitle":["The top 30 most popular emojis used about Lebron during the 2020 Playoffs"]  
    }).configure_axis(
    labelFontSize=16,
    titleFontSize=20
).configure_title(
    anchor='start',
    fontSize = 24,
    subtitleFontSize = 15
)

alt.Chart(...)

## **Let's dive in deep about the strike**

On 2020/8/25, the Milwaukee Bucks leave the court just before their scheduled playoff game to raise awareness about social justice issue (the shooting incident in Wisconsin). Let's see what people were talking about Lebron on Twitter during that period of players' strike.

In [16]:
df['Date'] = pd.to_datetime(df['date'])
mask = (df['Date'] > '2020-08-25 00:00') & (df['Date'] < '2020-08-28 23:59')
strike = df.loc[mask].sort_values('Date')
strike = strike.reset_index()
strike.drop(columns=['index','Date'],inplace=True)

strike.head()

,id,date,text,tags,emojis,clean_text,words,sentiment_score,day,hour,10min,min,POS,pos,neu,neg,NEG
0,1298260501088989184,2020-08-25 14:06:38+00:00,@CNN Hey @KingJames what about the people in H...,[],[],hey what about the people in hong kong arent...,"[hey, people, hong, kong, arent, scared, peopl...",0.242857,2020-08-25,14,00,06,0,0,1,0,0
1,1298260505933434886,2020-08-25 14:06:39+00:00,"@KingJames Lebron said, in reference to Jacob ...",[],"[🤦, 🏻, ♂]",lebron said in reference to jacob blake and k...,"[lebron, said, reference, jacob, blake, kenosh...",0.135714,2020-08-25,14,00,06,0,0,1,0,0
2,1298260506747142145,2020-08-25 14:06:39+00:00,Whitlock: LeBron James Is A Bigot https://t.co...,[],[],whitlock lebron james is a bigot,"[whitlock, lebron, james, bigot]",0.000000,2020-08-25,14,00,06,0,0,1,0,0
3,1298260517559996416,2020-08-25 14:06:42+00:00,@KingJames \nKingJames you should sue Lazy #Ja...,"[Jasonwhitlock, LeBronJames, LakeShow, Lakerna...",[],kingjames you should sue lazy for repeatedly...,"[kingjames, sue, lazy, repeatedly, using, name...",-0.375000,2020-08-25,14,00,06,0,0,0,1,0
4,1298260528217780232,2020-08-25 14:06:45+00:00,"@KingJames if you want to make change, but bod...",[],[],if you want to make change but body cams for ...,"[want, make, change, body, cams, every, police...",0.000000,2020-08-25,14,00,06,0,0,1,0,0


In [17]:
c=top_item(strike,'tags')
c

,tags,count
0,LeBronJames,379
1,BlackLivesMatter,281
2,NBA,244
3,BLM,159
4,JacobBlake,152
5,NBABoycott,125
6,NBAPlayoffs,120
7,nba,105
8,AffiliateLink,75
9,SportsCards,75


In [18]:
c1=c[:30]

alt.Chart(c1).mark_bar().encode(
    x=alt.X('tags',sort=['count'],title='Tags',axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count',title='Total Usage')
).properties(width=900,height=400,title={
      "text": ["Most frequent tags during Strike"],
      "subtitle":["The top 30 most used tags among the tweets about Lebron during NBA Players' Strike"]  
    }).configure_axis(
    labelFontSize=16,
    titleFontSize=20
).configure_title(
    anchor='start',
    fontSize = 24,
    subtitleFontSize = 15
)

alt.Chart(...)